In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/team_table",True)

Out[4]: True

Team Table: 

Team name along with group 

In [0]:
%sql
CREATE or REPLACE TABLE team_table (
    team_id INTEGER,
    team_name VARCHAR(20),
    group_name VARCHAR(1) 
);

In [0]:
%sql
--Wales as playoff A winers
--Poland as playoff B winers
--Kazakhstan as playoff C winers
INSERT INTO team_table (team_id,team_name,group_name) VALUES
-- Group A
    (1, 'Germany','A'),
    (2, 'Scotland','A'),
    (3, 'Hungary','A'),
    (4, 'Switzerland','A'),

-- Group B
    (5, 'Spain','B'),
    (6, 'Croatia','B'),
    (7, 'Italy','B'),
    (8, 'Albania','B'),

-- Group C
    (9, 'Slovenia','C'),
    (10, 'Denmark','C'),
    (11, 'Serbia','C'),
    (12, 'England','C'),

-- Group D
    (13, 'Netherlands','D'),
    (14, 'Austria','D'),
    (15, 'France','D'),
    (16, 'Wales','D'),

-- Group E
    (17, 'Belgium','E'),
    (18, 'Slovakia','E'),
    (19, 'Romania','E'),
    (20, 'Poland','E'),

-- Group F
    (21, 'Turkey','F'),
    (22, 'Kazakhstan','F'),
    (23, 'Portugal','F'),
    (24, 'Czech Republic','F');

num_affected_rows,num_inserted_rows
24,24


In [0]:
%sql
select * from team_table;


team_id,team_name,group_name
1,Germany,A
2,Scotland,A
3,Hungary,A
4,Switzerland,A
5,Spain,B
6,Croatia,B
7,Italy,B
8,Albania,B
9,Slovenia,C
10,Denmark,C


Probable Combinations for the Matches in the Round of 16 Match Stage

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/third_place_combination",True)

Out[9]: True

<img src="dbfs:/FileStore/tables/euro.html" alt="Euro Image">

In [0]:
%sql
CREATE or replace  TABLE third_place_combination(
  ID INTEGER,
  selected_groups VARCHAR(4),
  Team1B CHAR(1),
  Team1C CHAR(1),
  Team1E CHAR(1),
  Team1F CHAR(1)
);

INSERT INTO  third_place_combination (ID, selected_groups, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');

num_affected_rows,num_inserted_rows
15,15


Table for all the matches in EuroCup2024

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/match_table",True)

Out[11]: True

In [0]:
%sql
CREATE OR REPLACE TABLE match_table(
    match_id integer,
    match_fixtures VARCHAR(50),
    group_name VARCHAR(1),
    team1 VARCHAR(20),
    team2 VARCHAR(20),
    team1_goals INTEGER,
    team2_goals INTEGER,
    match_results VARCHAR(20),
    match_stage VARCHAR(20)
);

In [0]:
%sql
INSERT INTO match_table
SELECT
row_number() OVER (ORDER BY 1) as match_id,
concat(team1.team_name, ' Vs ' , team2.team_name )AS match_fixtures,
team1.group_name AS group_name,
team1.team_name AS team1,
team2.team_name AS team2,
cast(rand() * 6 AS INTEGER) AS goals_scored,
cast(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'Group Stage' AS match_stage
FROM
team_table AS team1
CROSS JOIN
team_table AS team2 
WHERE
team1.group_name = team2.group_name AND team1.team_id < team2.team_id;  

num_affected_rows,num_inserted_rows
36,36


In [0]:
%sql

-- Update match winner
UPDATE match_table SET match_results = 
CASE
WHEN team1_goals > team2_goals THEN team1
WHEN team1_goals < team2_goals THEN team2
ELSE 'DRAW'
END;

num_affected_rows
36


In [0]:
%sql
SELECT * FROM match_table ORDER BY match_id;


match_id,match_fixtures,group_name,team1,team2,team1_goals,team2_goals,match_results,match_stage
1,Germany Vs Switzerland,A,Germany,Switzerland,1,2,Switzerland,Group Stage
2,Germany Vs Hungary,A,Germany,Hungary,5,1,Germany,Group Stage
3,Germany Vs Scotland,A,Germany,Scotland,2,1,Germany,Group Stage
4,Scotland Vs Switzerland,A,Scotland,Switzerland,3,4,Switzerland,Group Stage
5,Scotland Vs Hungary,A,Scotland,Hungary,2,1,Scotland,Group Stage
6,Hungary Vs Switzerland,A,Hungary,Switzerland,2,1,Hungary,Group Stage
7,Spain Vs Albania,B,Spain,Albania,4,2,Spain,Group Stage
8,Spain Vs Italy,B,Spain,Italy,3,3,DRAW,Group Stage
9,Spain Vs Croatia,B,Spain,Croatia,0,5,Croatia,Group Stage
10,Croatia Vs Albania,B,Croatia,Albania,5,5,DRAW,Group Stage


Calculate point table based upon match table

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/point_table",True)

Out[16]: True

In [0]:
%sql
CREATE
OR REPLACE TABLE point_table AS
SELECT
  team_id,
  group_name,
  COUNT(*) AS matches_played,
  SUM (
    CASE
      WHEN goals_scored > goals_conceded THEN 1
      ELSE 0
    END
  ) AS wins,
  SUM (
    CASE
      WHEN goals_scored = goals_conceded THEN 1
      ELSE 0
    END
  ) AS draw,
  SUM(
    CASE
      WHEN goals_scored < goals_conceded THEN 1
      ELSE 0
    END
  ) AS lose,
  SUM(goals_scored) AS goals_scored,
  SUM(goals_conceded) AS goals_conceded,
  SUM(goals_scored) - SUM(goals_conceded) AS goal_diff,
  SUM(
    CASE
      WHEN goals_scored > goals_conceded THEN 3
      WHEN goals_scored = goals_conceded THEN 1
      ELSE 0
    END
  ) AS points
FROM(
    SELECT
      group_name,
      team1 AS team_id,
      team1_goals AS goals_scored,
      team2_goals AS goals_conceded
    FROM
      match_table
    UNION ALL
    SELECT
      group_name,
      team2 AS team_id,
      team2_goals AS goals_scored,
      team1_goals AS goals_conceded
    FROM
      match_table
  ) AS group_matches
GROUP BY
  team_id,
  group_name
ORDER BY
  group_name;

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from point_table;

team_id,group_name,matches_played,wins,draw,lose,goals_scored,goals_conceded,goal_diff,points
Germany,A,3,2,0,1,8,4,4,6
Scotland,A,3,1,0,2,6,7,-1,3
Hungary,A,3,1,0,2,4,8,-4,3
Switzerland,A,3,2,0,1,7,6,1,6
Italy,B,3,2,1,0,11,8,3,7
Croatia,B,3,1,1,1,12,9,3,4
Spain,B,3,1,1,1,7,10,-3,4
Albania,B,3,0,1,2,10,13,-3,1
Serbia,C,3,2,1,0,10,3,7,7
Denmark,C,3,1,1,1,8,10,-2,4


Create new table to store all the qualifying teams from each group

-TOP 2 teams from each group forwards to TOP 16

-TOP 4 teams that placed 3rd from all groups (4 out of 6)


In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/RO16_table",True)

Out[19]: False

In [0]:
%sql
CREATE OR REPLACE TABLE RO16_table AS
SELECT * FROM(
  SELECT team_id, group_name, points, goal_diff,
  row_number() OVER (PARTITION BY group_name ORDER BY points DESC, goal_diff DESC, goals_scored DESC, wins DESC)AS ranks
  FROM point_table
)AS top2_teams
WHERE ranks <= 2;

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from RO16_table;

team_id,group_name,points,goal_diff,ranks
Germany,A,6,4,1
Switzerland,A,6,1,2
Italy,B,7,3,1
Croatia,B,4,3,2
Serbia,C,7,7,1
England,C,4,0,2
Wales,D,9,5,1
Austria,D,6,3,2
Slovakia,E,9,3,1
Belgium,E,6,5,2


Top 4 teams from the 3 position team of every group 

Based upon points, goal_diff, goals_scored  and goals_conceded

In [0]:
%sql
INSERT INTO RO16_table
SELECT * FROM(
  SELECT team_id, group_name, points, goal_diff,
  row_number() OVER (PARTITION BY group_name ORDER BY points DESC, goal_diff DESC, goals_scored DESC, wins DESC) AS ranks
  FROM point_table
)AS top2_teams
WHERE ranks = 3 LIMIT 4;

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from RO16_table;

team_id,group_name,points,goal_diff,ranks
Germany,A,6,4,1
Switzerland,A,6,1,2
Italy,B,7,3,1
Croatia,B,4,3,2
Serbia,C,7,7,1
England,C,4,0,2
Wales,D,9,5,1
Austria,D,6,3,2
Slovakia,E,9,3,1
Belgium,E,6,5,2


Inserting Matches of Round of 16 into  match_table

Matches are based upond the combination of top 4 third teams 




In [0]:
%sql
INSERT INTO match_table (match_id, match_fixtures, group_name, team1, team2, team1_goals, team2_goals, match_results, match_stage)
WITH combination AS (
  SELECT concat_ws('', sort_array(collect_list(group_name)))AS group_combination
  FROM RO16_table
  WHERE ranks = 3
),
keys AS (
  SELECT * FROM third_place_combination
  WHERE selected_groups = (SELECT group_combination FROM combination)
)
SELECT
row_number() OVER (ORDER BY 1) + 36 AS match_id,
CONCAT(r1.team_id, ' VS ' ,r2.team_id)AS match_fixtures,
'N' AS group_name,
r1.team_id AS team1, 
r2.team_id AS team2, 
CAST(rand() * 6 AS INTEGER) AS goals_scored,
CAST(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'R16' AS match_stage
FROM RO16_table r1
JOIN RO16_table r2 ON r1.group_name = 'A' AND r1.ranks = 1 AND r2.group_name = 'C' AND r2.ranks = 2

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 37 AS match_id,
CONCAT(r1.team_id, ' VS ' ,r2.team_id)AS match_fixtures,
'N' AS group_name,
r1.team_id AS team1,
r2.team_id AS team2,
CAST(rand() * 6 AS INTEGER) AS goals_scored,
CAST(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'R16' AS match_stage
FROM RO16_table r1
JOIN RO16_table r2 ON r1.group_name = 'A' AND r1.ranks = 2 AND r2.group_name = 'B' AND r2.ranks = 2

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 38 AS match_id,
CONCAT(r1.team_id, ' VS ' ,r2.team_id)AS match_fixtures,
'N' AS group_name,
r1.team_id AS team1,
r2.team_id AS team2,
CAST(rand() * 6 AS INTEGER) AS goals_scored,
CAST(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'R16' AS match_stage
FROM RO16_table r1
JOIN RO16_table r2 ON r1.group_name = 'B' AND r1.ranks = 1 AND r2.group_name = (SELECT TEAM1B FROM keys) AND r2.ranks = 3

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 39 AS match_id,
CONCAT(r1.team_id, ' VS ' ,r2.team_id)AS match_fixtures,
'N' AS group_name,
r1.team_id AS team1,
r2.team_id AS team2,
CAST(rand() * 6 AS INTEGER) AS goals_scored,
CAST(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'R16' AS match_stage
FROM RO16_table r1
JOIN RO16_table r2 ON r1.group_name = 'C' AND r1.ranks = 1 AND r2.group_name = (select Team1C from keys) AND r2.ranks = 3

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 40 AS match_id,
CONCAT(r1.team_id, ' VS ' ,r2.team_id)AS match_fixtures,
'N' AS group_name,
r1.team_id AS team1, --home_team,
r2.team_id AS team2, --away_team,
CAST(rand() * 6 AS INTEGER) AS goals_scored,
CAST(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'R16' AS match_stage
FROM RO16_table r1
JOIN RO16_table r2 ON r1.group_name = 'F' AND r1.ranks = 1 AND r2.group_name = (select Team1F from keys) AND r2.ranks = 3

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 41 AS match_id,
CONCAT(r1.team_id, ' VS ' ,r2.team_id)AS match_fixtures,
'N' AS group_name,
r1.team_id AS team1, --home_team,
r2.team_id AS team2, --away_team,
CAST(rand() * 6 AS INTEGER) AS goals_scored,
CAST(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'R16' AS match_stage
FROM RO16_table r1
JOIN RO16_table r2 ON r1.group_name = 'D' AND r1.ranks = 2 AND r2.group_name = 'E' AND r2.ranks = 2

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 42 AS match_id,
CONCAT(r1.team_id, ' VS ' ,r2.team_id)AS match_fixtures,
'N' AS group_name,
r1.team_id AS team1,
r2.team_id AS team2,
CAST(rand() * 6 AS INTEGER) AS goals_scored,
CAST(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'R16' AS match_stage
FROM RO16_table r1
JOIN RO16_table r2 ON r1.group_name = 'D' AND r1.ranks = 1 AND r2.group_name = (SELECT Team1E from keys) AND r2.ranks = 3

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 43 AS match_id,
CONCAT(r1.team_id, ' VS ' ,r2.team_id)AS match_fixtures,
'N' AS group_name,
r1.team_id AS team1, 
r2.team_id AS team2, 
CAST(rand() * 6 AS INTEGER) AS goals_scored,
CAST(rand() * 6 AS INTEGER) AS goals_conceded,
'NONE' AS match_result,
'R16' AS match_stage
FROM RO16_table r1
JOIN RO16_table r2 ON r1.group_name = 'D' AND r1.ranks = 2 AND r2.group_name = 'F' AND r2.ranks = 2
;

num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
select * from match_table order by match_id;

match_id,match_fixtures,group_name,team1,team2,team1_goals,team2_goals,match_results,match_stage
1,Germany Vs Switzerland,A,Germany,Switzerland,1,2,Switzerland,Group Stage
2,Germany Vs Hungary,A,Germany,Hungary,5,1,Germany,Group Stage
3,Germany Vs Scotland,A,Germany,Scotland,2,1,Germany,Group Stage
4,Scotland Vs Switzerland,A,Scotland,Switzerland,3,4,Switzerland,Group Stage
5,Scotland Vs Hungary,A,Scotland,Hungary,2,1,Scotland,Group Stage
6,Hungary Vs Switzerland,A,Hungary,Switzerland,2,1,Hungary,Group Stage
7,Spain Vs Albania,B,Spain,Albania,4,2,Spain,Group Stage
8,Spain Vs Italy,B,Spain,Italy,3,3,DRAW,Group Stage
9,Spain Vs Croatia,B,Spain,Croatia,0,5,Croatia,Group Stage
10,Croatia Vs Albania,B,Croatia,Albania,5,5,DRAW,Group Stage


In [0]:
%sql
UPDATE match_table
SET match_results = 
CASE
WHEN team1_goals > team2_goals THEN team1
WHEN team1_goals < team2_goals THEN team2
ELSE
CASE
WHEN (team1_goals + 1) % 2 = 1 THEN team1
ELSE team1
END
END
WHERE match_stage = 'R16';

num_affected_rows
8


In [0]:
%sql
select * from match_table order by match_id;

match_id,match_fixtures,group_name,team1,team2,team1_goals,team2_goals,match_results,match_stage
1,Germany Vs Switzerland,A,Germany,Switzerland,1,2,Switzerland,Group Stage
2,Germany Vs Hungary,A,Germany,Hungary,5,1,Germany,Group Stage
3,Germany Vs Scotland,A,Germany,Scotland,2,1,Germany,Group Stage
4,Scotland Vs Switzerland,A,Scotland,Switzerland,3,4,Switzerland,Group Stage
5,Scotland Vs Hungary,A,Scotland,Hungary,2,1,Scotland,Group Stage
6,Hungary Vs Switzerland,A,Hungary,Switzerland,2,1,Hungary,Group Stage
7,Spain Vs Albania,B,Spain,Albania,4,2,Spain,Group Stage
8,Spain Vs Italy,B,Spain,Italy,3,3,DRAW,Group Stage
9,Spain Vs Croatia,B,Spain,Croatia,0,5,Croatia,Group Stage
10,Croatia Vs Albania,B,Croatia,Albania,5,5,DRAW,Group Stage


Quarter Finals

In [0]:
%sql
INSERT INTO match_table (match_id, match_fixtures, group_name, team1, team2, team1_goals, team2_goals, match_results, match_stage)
SELECT 
    Row_Number() OVER(ORDER BY 1) + 44 AS match_id,
    CONCAT(QF1.match_results, ' VS ', QF2.match_results) AS match_fixtures,
    'N' as group_name,
    QF1.match_results AS team1,
    QF2.match_results AS team2,
    CAST(rand() * 6 AS INT) AS goals_scored, 
    CAST(rand() * 6 AS INT) AS goals_conceded,
    'NONE' AS match_results,
    'Quarter-Final' AS match_stage
FROM 
    match_table QF1
JOIN 
    match_table QF2 ON 
        QF1.match_id = 39 AND QF2.match_id = 37

UNION ALL

SELECT 
    Row_Number() OVER(ORDER BY 1) + 45 AS match_id,
    CONCAT(QF1.match_results, ' VS ', QF2.match_results) AS match_fixtures,
    'N' as group_name,
    QF1.match_results AS team1,
    QF2.match_results AS team2,
    CAST(rand() * 6 AS INT) AS goals_scored, 
    CAST(rand() * 6 AS INT) AS goals_conceded,
    'NONE' AS match_results,
    'Quarter-Final' AS match_stage
FROM 
    match_table QF1
JOIN 
    match_table QF2 ON 
        QF1.match_id = 41 AND QF2.match_id = 42

UNION ALL

SELECT 
    Row_Number() OVER(ORDER BY 1) + 46 AS match_id,
    CONCAT(QF1.match_results, ' VS ', QF2.match_results) AS match_fixtures,
    'N' as group_name,
    QF1.match_results AS team1,
    QF2.match_results AS team2,
    CAST(rand() * 6 AS INT) AS goals_scored, 
    CAST(rand() * 6 AS INT) AS goals_conceded,
    'NONE' AS match_results,
    'Quarter-Final' AS match_stage
FROM 
    match_table QF1
JOIN 
    match_table QF2 ON 
        QF1.match_id = 43 AND QF2.match_id = 44

UNION ALL

SELECT 
    Row_Number() OVER(ORDER BY 1) + 47 AS match_id,
    CONCAT(QF1.match_results, ' VS ', QF2.match_results) AS match_fixtures,
    'N' as group_name,
    QF1.match_results AS team1,
    QF2.match_results AS team2,
    CAST(rand() * 6 AS INT) AS goals_scored, 
    CAST(rand() * 6 AS INT) AS goals_conceded,
    'NONE' AS match_results,
    'Quarter-Final' AS match_stage
FROM 
    match_table QF1
JOIN 
   match_table QF2 ON 
        QF1.match_id = 40 AND QF2.match_id = 38;

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from match_table order by match_id desc;

match_id,match_fixtures,group_name,team1,team2,team1_goals,team2_goals,match_results,match_stage
48,Serbia VS Croatia,N,Serbia,Croatia,0,1,NONE,Quarter-Final
47,Spain VS Austria,N,Spain,Austria,5,1,NONE,Quarter-Final
46,Czech Republic VS Austria,N,Czech Republic,Austria,2,5,NONE,Quarter-Final
45,Italy VS Germany,N,Italy,Germany,0,5,NONE,Quarter-Final
44,Austria VS Turkey,N,Austria,Turkey,4,0,Austria,R16
43,Wales VS Spain,N,Wales,Spain,0,5,Spain,R16
42,Austria VS Belgium,N,Austria,Belgium,5,4,Austria,R16
41,Czech Republic VS Denmark,N,Czech Republic,Denmark,2,0,Czech Republic,R16
40,Serbia VS Netherlands,N,Serbia,Netherlands,5,5,Serbia,R16
39,Italy VS Scotland,N,Italy,Scotland,3,3,Italy,R16


In [0]:
%sql
UPDATE match_table
SET match_results = 
CASE
WHEN team1_goals > team2_goals THEN team1
WHEN team1_goals < team2_goals THEN team2
ELSE
CASE
WHEN (team1_goals + 1) % 2 = 1 THEN team1
ELSE team1
END
END
WHERE match_stage = 'Quarter-Final';

num_affected_rows
4


In [0]:
%sql
select * from match_table order by match_id desc;

match_id,match_fixtures,group_name,team1,team2,team1_goals,team2_goals,match_results,match_stage
48,Serbia VS Croatia,N,Serbia,Croatia,0,1,Croatia,Quarter-Final
47,Spain VS Austria,N,Spain,Austria,5,1,Spain,Quarter-Final
46,Czech Republic VS Austria,N,Czech Republic,Austria,2,5,Austria,Quarter-Final
45,Italy VS Germany,N,Italy,Germany,0,5,Germany,Quarter-Final
44,Austria VS Turkey,N,Austria,Turkey,4,0,Austria,R16
43,Wales VS Spain,N,Wales,Spain,0,5,Spain,R16
42,Austria VS Belgium,N,Austria,Belgium,5,4,Austria,R16
41,Czech Republic VS Denmark,N,Czech Republic,Denmark,2,0,Czech Republic,R16
40,Serbia VS Netherlands,N,Serbia,Netherlands,5,5,Serbia,R16
39,Italy VS Scotland,N,Italy,Scotland,3,3,Italy,R16


SEMI FINAl

In [0]:
%sql
INSERT INTO match_table (match_id, match_fixtures,group_name, team1, team2, team1_goals, team2_goals, match_results, match_stage)
SELECT 
    Row_Number() OVER(ORDER BY 1) + 48 AS match_id,
    CONCAT(SF1.match_results, ' Vs. ', SF2.match_results) AS fixture,
    'N' as GroupName,
    SF1.match_results AS team1,
    SF2.match_results AS team2,
    CAST(rand() * 6 AS INT) AS goals_scored, 
    CAST(rand() * 6 AS INT) AS goals_conceded,
    'NONE' AS match_results,
    'Semi-Final' AS match_stage
FROM 
    match_table SF1
JOIN 
    match_table SF2 ON (
        (SF1.match_id = 45 AND SF2.match_id = 46) OR
        (SF1.match_id = 47 AND SF2.match_id = 48)
    );

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
UPDATE match_table
SET match_results = 
CASE
WHEN team1_goals > team2_goals THEN team1
WHEN team1_goals < team2_goals THEN team2
ELSE
CASE
WHEN (team1_goals + 1) % 2 = 1 THEN team1
ELSE team1
END
END
WHERE match_stage = 'Semi-Final';

num_affected_rows
2


In [0]:
%sql
select * from match_table order by match_id desc;

match_id,match_fixtures,group_name,team1,team2,team1_goals,team2_goals,match_results,match_stage
50,Spain Vs. Croatia,N,Spain,Croatia,3,1,Spain,Semi-Final
49,Germany Vs. Austria,N,Germany,Austria,5,4,Germany,Semi-Final
48,Serbia VS Croatia,N,Serbia,Croatia,0,1,Croatia,Quarter-Final
47,Spain VS Austria,N,Spain,Austria,5,1,Spain,Quarter-Final
46,Czech Republic VS Austria,N,Czech Republic,Austria,2,5,Austria,Quarter-Final
45,Italy VS Germany,N,Italy,Germany,0,5,Germany,Quarter-Final
44,Austria VS Turkey,N,Austria,Turkey,4,0,Austria,R16
43,Wales VS Spain,N,Wales,Spain,0,5,Spain,R16
42,Austria VS Belgium,N,Austria,Belgium,5,4,Austria,R16
41,Czech Republic VS Denmark,N,Czech Republic,Denmark,2,0,Czech Republic,R16




Final Match


In [0]:
%sql
INSERT INTO match_table (match_id, match_fixtures,group_name, team1, team2, team1_goals, team2_goals, match_results, match_stage)
SELECT 
    Row_Number() OVER(ORDER BY 1) + 52 AS match_id,
    CONCAT(F1.match_results, ' Vs. ', F2.match_results) AS fixture,
    'N' as GroupName,
    F1.match_results AS team1,
    F2.match_results AS Team2,
    CAST(rand() * 6 AS INT) AS goals_scored, 
    CAST(rand() * 6 AS INT) AS goals_conceded,
    'None' AS match_results,
    'Final' AS match_stage
FROM 
    match_table F1
JOIN 
    match_table F2 ON 
        F1.match_id = 49 AND F2.match_id = 50;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE match_table
SET match_results = 
    CASE
        WHEN team1_goals > team2_goals THEN team1
        WHEN team1_goals < team2_goals THEN team2
        ELSE 
          CASE 
                WHEN (team1_goals + 1) % 2 = 1 THEN team1
                ELSE team2
            END
    END
WHERE match_stage = 'Final';

num_affected_rows
1


In [0]:
%sql
select * from match_table order by match_id desc;

match_id,match_fixtures,group_name,team1,team2,team1_goals,team2_goals,match_results,match_stage
53,Germany Vs. Spain,N,Germany,Spain,5,0,Germany,Final
50,Spain Vs. Croatia,N,Spain,Croatia,3,1,Spain,Semi-Final
49,Germany Vs. Austria,N,Germany,Austria,5,4,Germany,Semi-Final
48,Serbia VS Croatia,N,Serbia,Croatia,0,1,Croatia,Quarter-Final
47,Spain VS Austria,N,Spain,Austria,5,1,Spain,Quarter-Final
46,Czech Republic VS Austria,N,Czech Republic,Austria,2,5,Austria,Quarter-Final
45,Italy VS Germany,N,Italy,Germany,0,5,Germany,Quarter-Final
44,Austria VS Turkey,N,Austria,Turkey,4,0,Austria,R16
43,Wales VS Spain,N,Wales,Spain,0,5,Spain,R16
42,Austria VS Belgium,N,Austria,Belgium,5,4,Austria,R16


Tie-Sheet Visualization

In [0]:
#import libraries
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Visualization").getOrCreate()

In [0]:
# Query from spark sql for different stages
all_teams_stats = spark.sql("SELECT * from point_table")
round_of_16 = spark.sql("""
                        SELECT 
    CASE WHEN match_id IN (39,37,41,42) THEN 'left' ELSE 'right' END AS side,
    match_id,
    team1, 
    team2, 
    team1_goals, 
    team2_goals 
FROM 
    match_table 
WHERE 
    match_id IN (39,37,41,42,43,44,40,38) 
    AND match_stage = 'R16' 
ORDER BY 
    CASE 
        WHEN match_id = 39 THEN 1
        WHEN match_id = 37 THEN 2
        WHEN match_id = 41 THEN 3
        WHEN match_id = 42 THEN 4
        WHEN match_id = 43 THEN 5
        WHEN match_id = 44 THEN 6
        WHEN match_id = 40 THEN 7
        WHEN match_id = 38 THEN 8
    END;

                        """)
quarter_final = spark.sql("SELECT team1, team2, team1_goals, team2_goals FROM match_table WHERE match_stage = 'Quarter-Final' ORDER BY match_id")
semi_final = spark.sql("SELECT team1, team2, team1_goals, team2_goals FROM match_table WHERE match_stage = 'Semi-Final' ORDER BY match_id DESC")
final = spark.sql("SELECT team1, team2, team1_goals, team2_goals FROM match_table WHERE match_stage = 'Final' ORDER BY match_id")

In [0]:
all_teams_stats_df = all_teams_stats.toPandas()
round_of_16_df = round_of_16.toPandas()
quarter_final_df = quarter_final.toPandas()
semi_final_df = semi_final.toPandas()
final_df = final.toPandas()

# Create table visualization DataFrame
table_visualization = pd.DataFrame({
    'Teams': all_teams_stats_df['team_id'],
    'Group': all_teams_stats_df['group_name'],
    'Pld': all_teams_stats_df['matches_played'],
    'W': all_teams_stats_df['wins'],
    'D': all_teams_stats_df['draw'],
    'L': all_teams_stats_df['lose'],
    'GF': all_teams_stats_df['goals_scored'],
    'GA': all_teams_stats_df['goals_conceded'],
    'GD': all_teams_stats_df['goal_diff'],
    'Pts':all_teams_stats_df['points']
})

# Create blank row as a dictionary to add one blank row for each group
blank_row = {
    'Teams': '', 
    'Group': '', 
    'Pld': '', 
    'W': '',
    'D': '',
    'L': '',
    'GF': '',
    'GA': '',
    'GD': '',
    'Pts': ''
}

groups = []
for i in range(0, len(table_visualization), 4):
    group = table_visualization.iloc[i:i+4]  
    groups.append(group) 
    if i < len(table_visualization) - 4:
        groups.append(pd.DataFrame([blank_row])) 

visualization_with_space = pd.concat(groups).reset_index(drop=True)
half_table_index = len(visualization_with_space) // 2
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
display(visualization_with_space)
print(half_table_index)

Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
Germany,A,3,2,0,1,8,4,4,6
Scotland,A,3,1,0,2,6,7,-1,3
Hungary,A,3,1,0,2,4,8,-4,3
Switzerland,A,3,2,0,1,7,6,1,6
,,,,,,,,,
Italy,B,3,2,1,0,11,8,3,7
Croatia,B,3,1,1,1,12,9,3,4
Spain,B,3,1,1,1,7,10,-3,4
Albania,B,3,0,1,2,10,13,-3,1
,,,,,,,,,


14


In [0]:
#Table information for the Group A,B,C
left_table = pd.DataFrame(columns=['Teams','Group','Pld','W','D','L','GF','GA','GD','Pts'])
left_table = pd.concat([left_table, pd.DataFrame([blank_row])], ignore_index=True)
left_table = pd.concat([left_table, visualization_with_space.iloc[:half_table_index]]).reset_index(drop=True)

#Table information for the group C,D,E
right_table = pd.DataFrame(columns=['Teams','Group','Pld','W','D','L','GF','GA','GD','Pts'])
right_table = pd.concat([right_table, visualization_with_space.iloc[half_table_index +1:]]).reset_index(drop=True)
right_table = pd.concat([pd.DataFrame([blank_row]), right_table], ignore_index=True)

display(left_table)
display(right_table)

Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
,,,,,,,,,
Germany,A,3,2,0,1,8,4,4,6
Scotland,A,3,1,0,2,6,7,-1,3
Hungary,A,3,1,0,2,4,8,-4,3
Switzerland,A,3,2,0,1,7,6,1,6
,,,,,,,,,
Italy,B,3,2,1,0,11,8,3,7
Croatia,B,3,1,1,1,12,9,3,4
Spain,B,3,1,1,1,7,10,-3,4
Albania,B,3,0,1,2,10,13,-3,1


Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
,,,,,,,,,
France,D,3,0,0,3,4,15,-11,0
Austria,D,3,2,0,1,13,10,3,6
Netherlands,D,3,1,0,2,12,9,3,3
Wales,D,3,3,0,0,13,8,5,9
,,,,,,,,,
Slovakia,E,3,3,0,0,11,8,3,9
Belgium,E,3,2,0,1,11,6,5,6
Romania,E,3,0,0,3,3,10,-7,0
Poland,E,3,1,0,2,6,7,-1,3


In [0]:
# Create  DataFrame for round of sixteen
visualize_r16 = pd.DataFrame(columns=['Round_of_16', 'rest'])

for _ in range(2):
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

for index, row in round_of_16_df.iterrows():
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['team1']} ({row['team1_goals']})"]})], ignore_index=True)
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['team2']} ({row['team2_goals']})"]})], ignore_index=True)

    for _ in range(2):
        visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

visualize_r16['rest'] = ''

splited_r16_index = len(visualize_r16) // 2
print(splited_r16_index)
display(visualize_r16)

17


Round_of_16,rest
,
,
Italy (3),
Scotland (3),
,
,
Germany (4),
England (1),
,
,


In [0]:
left_side_r16 = visualize_r16.iloc[:splited_r16_index + 1].reset_index(drop = True)
right_side_r16 = visualize_r16.iloc[splited_r16_index - 1:].reset_index(drop = True)
display(left_side_r16)
display(right_side_r16)

Round_of_16,rest
,
,
Italy (3),
Scotland (3),
,
,
Germany (4),
England (1),
,
,


Round_of_16,rest
,
,
Wales (0),
Spain (5),
,
,
Austria (4),
Turkey (0),
,
,


Quarter Final Visualization

In [0]:
# Dataframe for quarter final
visualize_qf = pd.DataFrame(columns=['quarter_final'])

for _ in range(4):
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'quarter_final': ['']})], ignore_index=True)

for index, row in quarter_final_df.iterrows():
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'quarter_final': [f"{row['team1']} ({row['team1_goals']})"]})], ignore_index=True)
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'quarter_final': [f"{row['team2']} ({row['team2_goals']})"]})], ignore_index=True)

    for _ in range(6):
        visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'quarter_final': ['']})], ignore_index=True)
visualize_qf['rest'] = ''
splited_qf_index = len(visualize_qf) // 2
display(visualize_qf)

quarter_final,rest
,
,
,
,
Italy (0),
Germany (5),
,
,
,
,


In [0]:
left_side_qf = pd.DataFrame(columns=['Quater_Final'])
left_side_qf = pd.concat([left_side_qf,visualize_qf.iloc[:splited_qf_index]]).reset_index(drop=True)
right_side_qf = pd.DataFrame(columns=['Quater_Final'])
right_side_qf = pd.concat([right_side_qf,visualize_qf.iloc[splited_qf_index-2:]]).reset_index(drop=True)


display(left_side_qf)
display(right_side_qf)

Quater_Final,quarter_final,rest
NaN,,
NaN,,
NaN,,
NaN,,
NaN,Italy (0),
NaN,Germany (5),
NaN,,
NaN,,
NaN,,
NaN,,


Quater_Final,quarter_final,rest
NaN,,
NaN,,
NaN,,
NaN,,
NaN,Spain (5),
NaN,Austria (1),
NaN,,
NaN,,
NaN,,
NaN,,


SemiFinal Visualization

In [0]:
visualize_sf = pd.DataFrame(columns=['semi_final'])

for _ in range(8):
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'semi_final': ['']})], ignore_index=True)

for index, row in semi_final_df.iterrows():
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'semi_final': [f"{row['team1']} ({row['team1_goals']})"]})], ignore_index=True)
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'semi_final': [f"{row['team2']} ({row['team2_goals']})"]})], ignore_index=True)
    for _ in range(6):
        visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'semi_final': ['']})], ignore_index=True)
visualize_sf['rest'] = ''
splited_sf_index = len(visualize_sf) // 2
print(splited_sf_index)
display(visualize_sf)


12


semi_final,rest
,
,
,
,
,
,
,
,
Spain (3),
Croatia (1),


In [0]:
left_side_sf = pd.DataFrame(columns=['Semi_Final'])
left_side_sf = pd.concat([left_side_sf, visualize_sf.iloc[:splited_sf_index]]).reset_index(drop=True)

for _ in range(5):
    left_side_sf = pd.concat([left_side_sf, pd.DataFrame({'Semi_Final': [''], 'rest': ['']})], ignore_index=True)

right_side_sf = pd.DataFrame({'Semi_Final': [''] * 7, 'rest': [''] * 7})
right_side_sf = pd.concat([right_side_sf, visualize_sf.iloc[splited_sf_index:]]).reset_index(drop=True)

display(left_side_sf)
display(right_side_sf)

Semi_Final,semi_final,rest
NaN,,
NaN,,
NaN,,
NaN,,
NaN,,
NaN,,
NaN,,
NaN,,
NaN,Spain (3),
NaN,Croatia (1),


Semi_Final,rest,semi_final
,,NaN
,,NaN
,,NaN
,,NaN
,,NaN
,,NaN
,,NaN
NaN,,
NaN,,
NaN,,



Final Match Visualization


In [0]:
visualize_final = pd.DataFrame(columns=['Final'])
for _ in range(6):
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)

for index,row in final_df.iterrows():
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['team1']} ({row['team1_goals']})"]})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['team2']} ({row['team2_goals']})"]})], ignore_index=True)

    if (index + 1) %2 == 0:
        visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)
        visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)
    for _ in range(2):
        visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)
visualize_final['space'] = ''
display(visualize_final)

Final,space
,
,
,
,
,
,
Germany (5),
Spain (0),
,
,


Merge Dataframes

In [0]:
left_side_qf.columns = [f"{col} q2" for col in left_side_qf.columns]

merged_df = pd.concat([left_side_r16, left_side_qf], axis=1)

left_side_sf.columns = [f"{col} s1" for col in left_side_sf.columns]
merged_df = pd.concat([merged_df, left_side_sf], axis=1)

visualize_final.columns = [f"{col} f" for col in visualize_final.columns]
merged_df = pd.concat([merged_df, visualize_final], axis=1)

right_side_sf.columns = [f"{col} s2" for col in right_side_sf.columns]

merged_df = pd.concat([merged_df, right_side_sf], axis=1)



right_side_qf.columns = [f"{col} q1" for col in right_side_qf.columns]
# Concatenate the DataFrames
merged_df = pd.concat([merged_df, right_side_qf], axis=1)

right_side_r16.columns = [f"{col} r2" for col in right_side_r16.columns]

# Concatenate the DataFrames
merged_df = pd.concat([merged_df,right_side_r16], axis=1)

left_table.columns = [f"{col} g1" for col in left_table.columns]
right_table.columns = [f"{col} g2" for col in right_table.columns]

# Concatenate the DataFrames
merged_df = pd.concat([left_table,merged_df, right_table], axis=1)

merged_df.fillna('', inplace=True)

display(merged_df)

Teamsg1,Groupg1,Pldg1,Wg1,Dg1,Lg1,GFg1,GAg1,GDg1,Ptsg1,Round_of_16,rest,Quater_Finalq2q2q2,quarter_finalq2q2q2,restq2q2q2,Semi_Finals1s1s1,semi_finals1s1s1,rests1s1s1,Finalfff,spacefff,Semi_Finals2s2s2,rests2s2s2,semi_finals2s2s2,Quater_Finalq1q1q1,quarter_finalq1q1q1,restq1q1q1,Round_of_16r2r2r2,restr2r2r2,Teamsg2,Groupg2,Pldg2,Wg2,Dg2,Lg2,GFg2,GAg2,GDg2,Ptsg2
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Germany,A,3,2,0,1,8,4,4,6,,,,,,,,,,,,,,,,,,,France,D,3,0,0,3,4,15,-11,0
Scotland,A,3,1,0,2,6,7,-1,3,Italy (3),,,,,,,,,,,,,,,,Wales (0),,Austria,D,3,2,0,1,13,10,3,6
Hungary,A,3,1,0,2,4,8,-4,3,Scotland (3),,,,,,,,,,,,,,,,Spain (5),,Netherlands,D,3,1,0,2,12,9,3,3
Switzerland,A,3,2,0,1,7,6,1,6,,,,Italy (0),,,,,,,,,,,Spain (5),,,,Wales,D,3,3,0,0,13,8,5,9
,,,,,,,,,,,,,Germany (5),,,,,,,,,,,Austria (1),,,,,,,,,,,,,
Italy,B,3,2,1,0,11,8,3,7,Germany (4),,,,,,,,Germany (5),,,,,,,,Austria (4),,Slovakia,E,3,3,0,0,11,8,3,9
Croatia,B,3,1,1,1,12,9,3,4,England (1),,,,,,,,Spain (0),,,,,,,,Turkey (0),,Belgium,E,3,2,0,1,11,6,5,6
Spain,B,3,1,1,1,7,10,-3,4,,,,,,,Spain (3),,,,,,,,,,,,Romania,E,3,0,0,3,3,10,-7,0
Albania,B,3,0,1,2,10,13,-3,1,,,,,,,Croatia (1),,,,,,,,,,,,Poland,E,3,1,0,2,6,7,-1,3
